In [11]:
import pickle 
import numpy as np
import os

In [12]:
def show_valid_replication(dataset, fair_coeff=1, target_label_idx=0, masked=False):
    
    fair_metrics = []
    fair_results = {}
    prefix = f'postprocess/evaluation-{dataset}/evaluation-{target_label_idx}'
    if masked: 
        prefix += '_masked'
    for i in range(1, 11):
        target_file = f'{prefix}/fair_eval_lambda={fair_coeff:.2f}_{i:04d}.pkl'
        if os.path.exists(target_file):
            fairs = pickle.load(open(target_file, 'rb'))
    
            if not fair_metrics:
                fair_metrics = list(fairs.keys())
            for met in fairs:
                if met not in fair_results:
                    fair_results[met] = {}
                for mod in fairs[met]:
                    if mod not in fair_results[met]:
                        fair_results[met][mod] = []
                    fair_results[met][mod].append(fairs[met][mod])
    
    perform_metrics = []
    perform_results = {}
    for i in range(1, 11):
        target_file = f'{prefix}/perform_eval_lambda={fair_coeff:.2f}_{i:04d}.pkl'
        if os.path.exists(target_file):
            performs = pickle.load(open(target_file, 'rb'))
            
            perform_models = list(performs.keys())
            
            if not perform_metrics:
                perform_metrics = list(performs[perform_models[0]].keys())

            for met in perform_metrics:
                if met not in perform_results:
                    perform_results[met] =  {}
                for mod in performs:
                    if mod not in perform_results[met]:
                        perform_results[met][mod] = []
                    
                    perform_results[met][mod].append(performs[mod][met])
            
    fair_metrics = list(fair_results.keys())
    fair_metrics_nested = {}
    fair_metrics_sorted = []
    should_add_eo = False
    for met_hparam in fair_metrics:
        met = met_hparam.split('_')[0]
        if met not in fair_metrics_nested:
            fair_metrics_nested[met] = []
        fair_metrics_nested[met].append(met_hparam)
    
    for met in ['constant', 'jaccard', 'indication']:
        if met in fair_metrics_nested:
            if len(fair_metrics_nested[met]) > 1:
                met_sorted = sorted(
                    fair_metrics_nested[met], key=lambda met: float(met.split('_')[-1]))
            else:
                met_sorted = fair_metrics_nested[met]
            fair_metrics_sorted += met_sorted
    fair_metrics = fair_metrics_sorted
#     print(fair_metrics)
    
    colnames = ' & ' + ' & '.join(fair_metrics)
    print(colnames + '\\\\')
    print('\\midrule')
    for met in fair_metrics:
        result = []
        for mod in fair_metrics:
            results = fair_results[met][mod]
            mean = np.mean(results, 0)[1]
            se = np.std(results, 0)[1] / np.sqrt(len(results))
#             result.append(f"${mean:.5f} \\pm {se:.5f}$")
            result.append(f"${mean:.5f}$")

        resultrow = met + ' & ' + ' & '.join(result)
        print(resultrow + '\\\\')

    
    for perform_metric in list(perform_results.keys()):
        result = []
        for mod in fair_metrics:
            results = perform_results[perform_metric][mod]
            mean = np.mean(results, 0)[1]
            se = np.std(results, 0)[1] / np.sqrt(len(results))
#             result.append(f"${mean:.5f} \\pm {se:.5f}$")
            result.append(f"${mean:.5f}$")
        resultrow = perform_metric + ' & ' + ' & '.join(result)
        print(resultrow + '\\\\')
    print('\\bottomrule')

In [13]:
show_valid_replication('adult', 0.1, 0)
print('\n' * 5)

show_valid_replication('adult', 0.1, 0, True)

 & constant & jaccard_0.01 & jaccard_1.0 & jaccard_5.0 & jaccard_10.0 & indication\\
\midrule
constant & $0.00947$ & $0.00948$ & $0.01175$ & $0.04236$ & $0.08087$ & $0.03878$\\
jaccard_0.01 & $0.00946$ & $0.00958$ & $0.01171$ & $0.04252$ & $0.08062$ & $0.03879$\\
jaccard_1.0 & $0.01080$ & $0.01095$ & $0.01291$ & $0.04317$ & $0.08098$ & $0.03996$\\
jaccard_5.0 & $0.03083$ & $0.03101$ & $0.03168$ & $0.05748$ & $0.09345$ & $0.05586$\\
jaccard_10.0 & $0.05380$ & $0.05394$ & $0.05453$ & $0.07573$ & $0.10958$ & $0.07678$\\
indication & $0.05648$ & $0.05706$ & $0.05769$ & $0.07750$ & $0.11243$ & $0.08001$\\
ACC & $0.04340$ & $0.04312$ & $0.04273$ & $0.04284$ & $0.04273$ & $0.04313$\\
HA & $0.92550$ & $0.92551$ & $0.92548$ & $0.92551$ & $0.92547$ & $0.92548$\\
ebF1 & $0.61649$ & $0.61641$ & $0.61651$ & $0.61647$ & $0.61655$ & $0.61621$\\
maF1 & $0.22600$ & $0.22598$ & $0.22573$ & $0.22600$ & $0.22613$ & $0.22581$\\
miF1 & $0.61668$ & $0.61690$ & $0.61687$ & $0.61695$ & $0.61702$ & $0.61672$\\


In [23]:
show_valid_replication('adult', 10, 0)
print('\n' * 5)

show_valid_replication('adult', 10, 0, True)

 & constant & jaccard_0.01 & jaccard_1.0 & jaccard_5.0 & jaccard_10.0 & indication\\
\midrule
constant & $0.00882$ & $0.00861$ & $0.01006$ & $0.03792$ & $0.07509$ & $0.03857$\\
jaccard_0.01 & $0.00883$ & $0.00873$ & $0.01007$ & $0.03792$ & $0.07570$ & $0.03848$\\
jaccard_1.0 & $0.01037$ & $0.01018$ & $0.01127$ & $0.03858$ & $0.07571$ & $0.03946$\\
jaccard_5.0 & $0.03066$ & $0.03073$ & $0.03104$ & $0.05361$ & $0.08755$ & $0.05624$\\
jaccard_10.0 & $0.05429$ & $0.05397$ & $0.05293$ & $0.07167$ & $0.10449$ & $0.07826$\\
indication & $0.05709$ & $0.05767$ & $0.05626$ & $0.07414$ & $0.10708$ & $0.08055$\\
ACC & $0.04306$ & $0.04315$ & $0.04349$ & $0.04265$ & $0.04319$ & $0.04314$\\
HA & $0.92545$ & $0.92547$ & $0.92552$ & $0.92550$ & $0.92549$ & $0.92551$\\
ebF1 & $0.61630$ & $0.61645$ & $0.61649$ & $0.61649$ & $0.61650$ & $0.61649$\\
maF1 & $0.22605$ & $0.22600$ & $0.22597$ & $0.22639$ & $0.22598$ & $0.22581$\\
miF1 & $0.61683$ & $0.61689$ & $0.61713$ & $0.61707$ & $0.61688$ & $0.61690$\\


In [18]:
show_valid_replication('credit', 10, 0)

print('\n' * 5)

show_valid_replication('credit', 10, 0, True)

 & constant & jaccard_0.01 & jaccard_1.0 & jaccard_5.0 & jaccard_10.0 & indication\\
\midrule
constant & $0.34755$ & $0.34692$ & $0.34907$ & $0.35614$ & $0.34529$ & $0.38961$\\
jaccard_0.01 & $0.35401$ & $0.35358$ & $0.35678$ & $0.34968$ & $0.34481$ & $0.39550$\\
jaccard_1.0 & $0.35190$ & $0.34531$ & $0.35474$ & $0.35031$ & $0.35571$ & $0.39685$\\
jaccard_5.0 & $0.45084$ & $0.45079$ & $0.45428$ & $0.44899$ & $0.44816$ & $0.50280$\\
jaccard_10.0 & $0.52896$ & $0.52689$ & $0.52946$ & $0.53610$ & $0.51956$ & $0.57879$\\
indication & $0.41724$ & $0.41597$ & $0.42097$ & $0.42632$ & $0.42725$ & $0.46687$\\
ACC & $0.11333$ & $0.11167$ & $0.11200$ & $0.11167$ & $0.11167$ & $0.11333$\\
HA & $0.77747$ & $0.77727$ & $0.77820$ & $0.77737$ & $0.77757$ & $0.77757$\\
ebF1 & $0.55603$ & $0.55701$ & $0.55644$ & $0.55617$ & $0.55634$ & $0.55559$\\
maF1 & $0.40960$ & $0.41072$ & $0.40940$ & $0.40939$ & $0.41091$ & $0.40836$\\
miF1 & $0.56413$ & $0.56403$ & $0.56437$ & $0.56341$ & $0.56403$ & $0.56338$\\
